In [275]:
import os
from crewai import Agent, Task, Crew
from dotenv import load_dotenv, find_dotenv
from crewai.tools import BaseTool
from tavily import TavilyClient
from langchain_openai import ChatOpenAI
from langchain.tools import tool

In [276]:
load_dotenv(find_dotenv())

llm = ChatOpenAI(model = "gpt-3.5-turbo-0125")

# Agentes
- Pesquisador de Mercado
- Analista de Tendências
- Criador de Post

## Pesquisador de Mercado

In [277]:
pesquisador = Agent(
    role = "Pesquisador de Mercado",
    goal = "Coletar e Organizar informações relevantes sobre {sector}",
    backstory = """
        Você é um pesquisador experiente que analisa tendências de mercado
        e coleta dados relevantes sobre {sector}. Seu trabalho é garantir que 
        todas as informações estejam atualizadas e bem documentadas.
    """,
    allow_delegation = False,
    verbose = True
)


In [278]:
coleta_dados = Task(
    description=(
        "1. Pesquisar e coletar informações atualizadas sobre {sector}."
        "2. identificar os principasi players, tendências e estatísticas do setor."
        "3. Organizar os dados de forma clara para análise."
    ),
    expected_output="um documento estruturado contendo dados de mercado sobre {sector}.",
    agent = pesquisador
)   

## Analista de notícias

In [279]:
tavily = TavilyClient(api_key="TAVILY_API_KEY")

In [280]:
@tool
def buscar_noticias(termo: str) -> str:
    """Busca notícias recentes com Tavily e retorna os títulos e links."""
    resultados = tavily.search(query=termo, search_depth="advanced", include_answer=False)
    noticias = resultados.get("results", [])
    
    if not noticias:
        return "Nenhuma notícia encontrada."

    resposta = ""
    for item in noticias[:5]:  # Limita a 5 resultados
        resposta += f"- {item['title']}\n  {item['url']}\n\n"

    return resposta

In [281]:
analista = Agent(
    role = "Pesquisador de notícias",
    goal = "Buscar as últimas notícias sobre um assunto específico",
    backstory = "Você é um expert em buscar informações recentes na internet.",
    allow_delegation = False,
    verbose = True
)

In [282]:
analise_noticias = Task(
    description = "Encontre notícias sobre Agentes de IA no ano de 2025",
    expected_output="Resumo com links das notícias mais relevantes.",
    tool = [buscar_noticias],
    agent = analista
)

## Criador de Post

In [283]:

redator = Agent(
    role = "Criador de conteúdo digital",
    goal = "Elaborar um post para o Linkedin para o setor {sector} de forma descontraída e fácil compreensão",
    backstory = """
        Você é um redator de mídias digitais profissional que transforma análises de mercado e de notícias em um post
        de fácil compreensão para usuários de mídias sociais.
    """,
    allow_delegation = False,
    verbose = True
)

In [284]:
redacao_relatorio = Task(
    description = (
        "1. usar a análise de notícias para criar um post Linkedin detalhado sobre {sector}, não precisando colocar o link da referência."
        "2. Garantir que o post seja bem estruturado e compreensível e de fácil leitura."
    ),
    expected_output="Um post de fácil compreensão e não muito grande para leitores de mídias sociais como o Linkedin, utilizando de emojis sempre que possível. Comece o texto com: 'Você sabe o que é Agente de IA', para gerar curiosidade",
    context=[analise_noticias, coleta_dados],
    agent = redator

)

In [285]:
crew = Crew(
    agents = [pesquisador, analista, redator],
    tasks = [coleta_dados, analise_noticias, redacao_relatorio],
    verbose = True,
    manager_llm = llm
)

In [286]:
resultado = crew.kickoff(inputs={"sector": "Agentes de IA"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5903a99f-e0fb-42c4-bec8-260a337ccee7                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: e3513e30-8f8d-4711-9d6e-b018431899fd
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: e3513e30-8f8d-4711-9d6e-b018431899fd
       Status: Executing Task...
    └── 🤖 Agent: Pesquisador de Mercado
            Status: In Progress

# Agent: Pesquisador de Mercado
## Task: 1. Pesquisar e coletar informações atualizadas sobre Agentes de IA.2. identificar os principasi players, tendências e estatísticas do setor.3. Organizar os dados de forma clara para análise.


# Agent: Pesquisador de Mercado
## Final Answer: 
# Documento Estruturado sobre Agentes de IA

## 1. Introdução
Agentes de IA (Inteligência Artificial) são sistemas computacionais que podem perceber seu ambiente, tomar decisões e realizar ações baseadas em aprendizados prévios. Com o crescimento do interesse em IA, surgiu uma variedade de aplicações, desde assistentes pessoais até sistemas complexos de aprendizado automático.

## 2. Principais Players no Mercado de Agentes de IA
### 2.1. Grandes Empresas
- **Google**: Líder em pesquisa de machine learning, especialmente com a plataforma TensorFlow, e no desenvolvimento de assistentes inteligentes como o Google Assistant.
- **Microsoft**: Oferece uma variedade de serviços de IA por meio de sua plataforma Az

🚀 Crew: crew
└── 📋 Task: e3513e30-8f8d-4711-9d6e-b018431899fd
       Status: Executing Task...
    └── 🤖 Agent: Pesquisador de Mercado
            Status: ✅ Completed

...

# Agent: Criador de conteúdo digital
## Task: 1. usar a análise de notícias para criar um post Linkedin detalhado sobre Agentes de IA, não precisando colocar o link da referência.2. Garantir que o post seja bem estruturado e compreensível e de fácil leitura.


# Agent: Criador de conteúdo digital
## Final Answer: 
Você sabe o que é Agente de IA? 🤖✨ 

Em 2025, os Agentes de Inteligência Artificial se tornaram verdadeiras estrelas no mercado, revolucionando a maneira como trabalhamos e interagimos com a tecnologia! 🚀

Esses agentes têm um papel fundamental em otimizar a força de trabalho, automatizando tarefas repetitivas e liberando os humanos para se concentrarem em atividades mais estratégicas. Imagine o quanto sua equipe poderia produzir se pudesse contar com um assistente que faz o trabalho “pesado”! 💪🧠

Recentemente, as inovações em Processamento de Linguagem Natural (NLP) melhoraram a comunicação entre humanos e máquinas. Isso significa que nossos assistentes de IA agora entendem 

🚀 Crew: crew
├── 📋 Task: e3513e30-8f8d-4711-9d6e-b018431899fd
│      Assigned to: Pesquisador de Mercado
│      Status: ✅ Completed
│   └── 🤖 Agent: Pesquisador de Mercado
│           Status: ✅ Completed
├── 📋 Task: 72ed8a39-dcae-426e-8daf-e57da260ff5c
│      Assigned to: Pesquisador de notícias
│      Status: ✅ Completed
│   └── 🤖 Agent: Pesquisador de notícias
│           Status: ✅ Completed
└── 📋 Task: 8dfd25b0-e32c-4636-b983-00c0d7762ac5
       Status: Executing Task...
    └── 🤖 Agent: Criador de conteúdo digital
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: e3513e30-8f8d-4711-9d6e-b018431899fd
│      Assigned to: Pesquisador de Mercado
│      Status: ✅ Completed
│   └── 🤖 Agent: Pesquisador de Mercado
│           Status: ✅ Completed
├── 📋 Task: 72ed8a39-dcae-426e-8daf-e57da260ff5c
│      Assigned to: Pesquisador de notícias
│      Status: ✅ Completed
│   └── 🤖 Agent: Pesquisador de notícias
│           Status: ✅ Completed
└── 📋 Task: 8dfd25b0-e32c-4636-b983-00c0d7762ac5
       Assigned to: Criador de conteúdo digital
       Status: ✅ Completed
    └── 🤖 Agent: Criador de conteúdo digital
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8dfd25b0-e32c-4636-b983-00c0d7762ac5                                                                     │
│  Agent: Criador de conteúdo digital                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5903a99f-e0fb-42c4-bec8-260a337ccee7                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [287]:
display(Markdown(str(resultado)))

Você sabe o que é Agente de IA? 🤖✨ 

Em 2025, os Agentes de Inteligência Artificial se tornaram verdadeiras estrelas no mercado, revolucionando a maneira como trabalhamos e interagimos com a tecnologia! 🚀

Esses agentes têm um papel fundamental em otimizar a força de trabalho, automatizando tarefas repetitivas e liberando os humanos para se concentrarem em atividades mais estratégicas. Imagine o quanto sua equipe poderia produzir se pudesse contar com um assistente que faz o trabalho “pesado”! 💪🧠

Recentemente, as inovações em Processamento de Linguagem Natural (NLP) melhoraram a comunicação entre humanos e máquinas. Isso significa que nossos assistentes de IA agora entendem e respondem a nossas perguntas de forma mais natural e eficiente, aumentando a satisfação do cliente e a produtividade das operações. 🌟🤝

Porém, não podemos esquecer das questões éticas que surgem com o uso crescente da IA. 🤔 É essencial que empresas adotem práticas transparentes e responsáveis. Afinal, como podemos confiar em um agente de IA se não sabemos como ele toma decisões? A regulamentação está no horizonte e precisa ser uma prioridade!

2025 também é visto como o ano em que os Agentes de IA se tornaram comuns em diversas indústrias, de atendimento ao cliente a marketing. As empresas estão se adaptando rapidamente e colhendo os frutos dessa revolução tecnológica! 🌏💼

E o que vem pela frente? As tendências indicam que a personalização das interações com os usuários só tende a crescer, impulsionada por tecnologias emergentes como o 5G. Estamos apenas começando a explorar o potencial desses agentes incríveis no nosso dia a dia. 🚀🔮

Para quem está no setor de tecnologia ou simplesmente interessado no futuro dos negócios, ficar de olho nesse panorama é essencial. Agentes de IA não são apenas uma tendência passageira; eles estão aqui para ficar e realmente transformar nossa maneira de trabalhar. 

Vamos juntos nessa jornada? 💡🔗 #InteligenciaArtificial #AgentesDeIA #Inovação #Tecnologia #ÉticaNaIA